In [2]:
import pandas as pd
import numpy as np

In [115]:
df = pd.read_excel('/Users/heemin/Documents/DHLab/mv/project/label.xlsx')
df.head()

,file,label_JH,label_HM,label_YG
0,HJY_0,angry,Negative,Angry
1,HJY_1,angry,Negative,Angry
2,HJY_2,angry,Negative,Angry
3,HJY_3,neutral,Neutral,Neutral
4,HJY_4,neutral,Neutral,Neutral


In [116]:
df['label1'] = df['label_JH']
df['label2'] = df['label_HM']
df['label3'] = df['label_YG']

In [117]:
df['label_JH'].unique()

array(['angry', 'neutral', 'happy', 'surprise', nan], dtype=object)

In [118]:
df['label_YG'].unique()

array(['Angry', 'Neutral', 'Happy', 'Surprise'], dtype=object)

In [119]:
df['label_HM'].unique()

array(['Negative', 'Neutral', 'Positive', nan], dtype=object)

In [120]:
df.loc[df['label_JH']=='angry', 'label1'] = 'Negative'
df.loc[df['label_JH']=='surprise', 'label1'] = 'Positive'
df.loc[df['label_JH']=='happy', 'label1'] = 'Positive'
df.loc[df['label_JH']=='neutral', 'label1'] = 'Neutral'

df.loc[df['label_YG']=='Angry', 'label3'] = 'Negative'
df.loc[df['label_YG']=='Surprise', 'label3'] = 'Positive'
df.loc[df['label_YG']=='Happy', 'label3'] = 'Positive'

In [121]:
df2 = df[['file','label1','label2', 'label3']]
df2.isna().sum()

file        0
label1      4
label2    113
label3      0
dtype: int64

In [122]:
df2_dropna=df2.dropna()
df2_dropna['true'] = df2_dropna.mode(axis=1)

/var/folders/sc/sg8zc8mj6630dt6_wh5ffjvr0000gn/T/ipykernel_31548/2715575546.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_dropna['true'] = df2_dropna.mode(axis=1)


In [123]:
df2_dropna

,file,label1,label2,label3,true
0,HJY_0,Negative,Negative,Negative,Negative
1,HJY_1,Negative,Negative,Negative,Negative
2,HJY_2,Negative,Negative,Negative,Negative
3,HJY_3,Neutral,Neutral,Neutral,Neutral
4,HJY_4,Neutral,Neutral,Neutral,Neutral
...,...,...,...,...,...
627,LTW_80,Neutral,Neutral,Neutral,Neutral
628,LTW_81,Neutral,Neutral,Neutral,Neutral
629,LTW_82,Neutral,Neutral,Neutral,Neutral
630,LTW_83,Neutral,Neutral,Neutral,Neutral


In [124]:
df2_dropna.to_excel('/Users/heemin/Documents/DHLab/mv/project/label_true.xlsx')

## data crop

In [5]:
import os
import natsort

from collections import Counter

In [265]:
fl = os.listdir('/Users/heemin/Documents/DHLab/mv/project/csv')

In [266]:
file_list = natsort.natsorted(fl)

In [272]:
drop_file =[]
for file in file_list:
    df1 = pd.read_csv('/Users/heemin/Documents/DHLab/mv/project/csv/'+file)[['total_count', 'emotion']]
    if len(df1) == 0:
        drop_file.append(file)

In [359]:
true_label = pd.read_excel('/Users/heemin/Documents/DHLab/mv/project/label_true.xlsx')
drop_filename = []
for file in drop_file:
    drop_filename.append(file.split('.')[0])
drop_df = pd.DataFrame(drop_filename, columns=['file'])
drop_df['bad'] = 1
label1 = pd.merge(true_label, drop_df, how='left', on='file')
label2 = label1[label1['bad'].isnull()].iloc[:,1:-1]

In [360]:
label2['Neutral'] = 0
label2.loc[label2['true'] == 'Neutral', 'Neutral'] = 1
label2 = label2.reset_index()

In [98]:
for idx in range(len(label2)):
    file = label2['file'][idx]
    df1 = pd.read_csv('/Users/heemin/Documents/DHLab/mv/project/csv/'+file+'.mp4.csv')[['total_count', 'emotion']]
    end_frame = int(df1['total_count'][-1:])
    df2 = pd.merge(pd.DataFrame([i for i in range(1, end_frame+1)], columns=['total_count']), df1, how='outer').fillna('Neutral')
    if label2['Neutral'][idx] == 0:
        split = 0
        frame2scound = []
        crop_frame_start = 0
        for i in range(1, end_frame+1):
            if i % 25 == 0 or i == end_frame+1:
                emolist = df2['emotion'][split:i].to_list()
                emocount = Counter(emolist)
                top3 = emocount.most_common(n=3)
                if len(top3) == 0: # top3 감정이 없음 = 해당 1초에 감정이 인식되지 않음
                    crop_frame_start = i
                elif len(top3) == 1:
                    if top3[0][0] == 'Neutral':
                        crop_frame_start = i
                    else:
                        continue
                else:
                    continue
                split += 25

        if end_frame%25 >0:
            r_end_frame = end_frame//25*25+25
        else:
            r_end_frame = end_frame//25*25
        crop_frame_end = r_end_frame
        reverse_df2 = pd.merge(pd.DataFrame([i for i in range(r_end_frame, -1, -1)], columns=['total_count']), df1, how='outer').fillna('Neutral')
        split = 0
        for i in range(1, end_frame+1):
            if i % 25 == 0 or i == end_frame+1:
                emolist = reverse_df2['emotion'][split:i].to_list()
                emocount = Counter(emolist)
                top3 = emocount.most_common(n=3)
                if len(top3) == 0: # top3 감정이 없음 = 해당 1초에 감정이 인식되지 않음
                    crop_frame_end = r_end_frame - i
                elif len(top3) == 1:
                    if top3[0][0] == 'Neutral':
                        crop_frame_end = r_end_frame - i
                    else:
                        continue
                else:
                    continue
                split += 25
        if crop_frame_start == crop_frame_end:
            crop_frame_start = 0
            crop_frame_end = r_end_frame
        df3 = df2[crop_frame_start:crop_frame_end]
        print(file, crop_frame_start, crop_frame_end)
        if 'KSY_57' in file:
            break
    else:
        df3 = df2
    df3.to_csv('/Users/heemin/Documents/DHLab/mv/project/csv_crop/'+file+'.mp4.csv')


HJY_0 0 50
HJY_1 0 50
HJY_2 0 150
HJY_5 25 225
HJY_6 0 100
HJY_7 0 225
HJY_11 0 50
HJY_22 0 250
HJY_23 0 125
HJY_24 0 50
HJY_25 25 250
HJY_27 0 100
HJY_28 0 500
HJY_29 0 250
HJY_30 0 150
HJY_40 0 725
HJY_41 0 50
HJY_42 0 100
HJY_58 0 550
HJY_64 0 100
KSY_21 0 25
KSY_22 0 50
KSY_23 0 50
KSY_24 0 50
KSY_25 0 50
KSY_26 0 50
KSY_27 0 150
KSY_28 0 150
KSY_29 0 50
KSY_45 0 225
KSY_46 0 150
KSY_47 25 125
KSY_48 0 75
KSY_49 0 100
KSY_50 0 275
KSY_51 0 125
KSY_52 0 275
KSY_53 0 425
KSY_54 0 200
KSY_55 0 225
KSY_56 0 225
KSY_57 25 325


In [ ]:
df2[0:50]

In [103]:
df3[0:50]

,total_count,emotion
25,26,Neutral
26,27,Neutral
27,28,Neutral
28,29,Happy
29,30,Happy
30,31,Neutral
31,32,Neutral
32,33,Neutral
33,34,Neutral
34,35,Neutral


In [282]:
reverse_df2.T

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
total_count,75,74,73,72,71,70,69,68,67,66,...,9,8,7,6,5,4,3,2,1,0
emotion,Neutral,Neutral,Neutral,Happy,Happy,Happy,Neutral,Neutral,Neutral,Neutral,...,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral


In [8]:
emotion_dict = {'Angry':'Negative', 'Disgust':'Negative', 'Fear':'Negative', 'Happy':'Positive', 
    'Sad':'Negative', 'Suprise':'Positive', 'Neutral':'Neutral'}

In [327]:
# pd.set_option('display.max_rows', None)
# pd.options.display.max_rows = 60

In [363]:
len(label2)

434

In [404]:
import torch

In [405]:
w = torch.empty(3, 5)

In [406]:
w

tensor([[0.0000e+00, 0.0000e+00, 2.8014e+25, 1.4013e-45, 1.2612e-44],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 3.6013e-43],
        [1.8754e+28, 4.0611e-08, 1.0632e-05, 1.2812e-11, 8.5018e-07]])

In [424]:
k = torch.tensor(20)

In [425]:
k

tensor(20)

In [427]:
j = torch.nn.init.zeros_(k)
j

tensor(0)

In [429]:
j += k*0.0125

RuntimeError: result type Float can't be cast to the desired output type Long

In [104]:
label2 = pd.read_excel('/Users/heemin/Documents/DHLab/mv/project/ll.xlsx')

In [123]:
threshold = 0.5
# p10_len = 5
for n in range(1, 6):
    n_total = 0
    n_correct = 0 
    e_total = 0
    e_correct = 0
    n_video_total = 0
    n_video_correct = 0
    e_video_total = 0
    e_video_correct = 0
    p10_len = n
    for i in range(len(label2)):
        file = label2['file'][i]
        df = pd.read_csv('/Users/heemin/Documents/DHLab/mv/project/csv_crop/'+file+'.mp4.csv', index_col=0)
        label = label2.loc[label2['file'] == file]['true'][i]

        if label2['Neutral'][i] == 0:
            l1 = df['total_count'].tolist()
            l2 = df['emotion'].tolist()
            df1 = pd.DataFrame([l1,l2]).T
            a = list(range(l1[-1]+1))
            index_df = pd.DataFrame(a)
            df2 = pd.merge(index_df, df1, how="outer")
            df2 = df2.fillna('none')
            l3 = []
            split = 0
            for i in range(len(df2)):
                if i % 25 == 0 and i != 0 or i == len(df2): # 25 frame 마다 연산
                    emolist = df2[1][split:i].to_list()
                    emocount = Counter(emolist)
                    top3 = emocount.most_common(n=3)

                    if top3[0][0] == 'none':
                        if len(top3) == 1:
                            l3.append([i, 'Neutral'])
                        elif top3[1][0] == 'Neutral':
                            if len(top3) == 3:
                                if top3[2][1] >= p10_len:
                                    l3.append([i, top3[2][0]])
                                else:
                                    l3.append([i, top3[1][0]])
                            else:
                                l3.append([i, top3[1][0]])
                    elif top3[0][0] == 'Neutral':
                        if len(top3) == 1:
                            l3.append([i, 'Neutral'])
                        elif top3[1][0] == 'none':
                            if len(top3) == 2: 
                                l3.append([i, 'Neutral'])
                            else:
                                if top3[2][1] >= p10_len:
                                    l3.append([i, top3[2][0]])
                                else:
                                    l3.append([i, top3[0][0]])
                        else:
                            if top3[1][1] >= p10_len:
                                l3.append([i, top3[1][0]])
                            else:
                                l3.append([i, top3[0][0]])
                    else:
                        l3.append([i, top3[0][0]])
                    split+=25
            # pd.DataFrame(l3, columns=[['count','emotion']]).to_csv('/Users/heemin/Documents/DHLab/mv/project/dir/emotion_top3/'+j[1])
            video_second_total = 0
            video_second_correct = 0
            for scound, emotion in l3:
                video_second_total += 1
                e_total+=1
                if emotion_dict[emotion] == label:
                    e_correct += 1
                    video_second_correct += 1
                # print(file, emotion, label)

            video_second_acc = video_second_correct / video_second_total
            # print(video_second_acc)
            e_video_total += 1
            if video_second_acc >= threshold:
                e_video_correct+=1
            
        else:
            l1 = df['total_count'].tolist()
            l2 = df['emotion'].tolist()
            df1 = pd.DataFrame([l1,l2]).T
            a = list(range(l1[-1]+1))
            index_df = pd.DataFrame(a)
            df2 = pd.merge(index_df, df1, how="outer")
            df2 = df2.fillna('none')
            l3 = []
            split = 0
            for i in range(len(df2)):
                if i % 25 == 0 and i != 0 or i == len(df2): # 25 frame 마다 연산
                    emolist = df2[1][split:i].to_list()
                    emocount = Counter(emolist)
                    top3 = emocount.most_common(n=3)

                    if top3[0][0] == 'none':
                        if len(top3) == 1:
                            l3.append([i, 'Neutral'])
                        elif top3[1][0] == 'Neutral':
                            if len(top3) == 3:
                                if top3[2][1] >= p10_len:
                                    l3.append([i, top3[2][0]])
                                else:
                                    l3.append([i, top3[1][0]])
                            else:
                                l3.append([i, top3[1][0]])
                    elif top3[0][0] == 'Neutral':
                        if len(top3) == 1:
                            l3.append([i, 'Neutral'])
                        elif top3[1][0] == 'none':
                            if len(top3) == 2: 
                                l3.append([i, 'Neutral'])
                            else:
                                if top3[2][1] >= p10_len:
                                    l3.append([i, top3[2][0]])
                                else:
                                    l3.append([i, top3[0][0]])
                        else:
                            if top3[1][1] >= p10_len:
                                l3.append([i, top3[1][0]])
                            else:
                                l3.append([i, top3[0][0]])
                    else:
                        l3.append([i, top3[0][0]])
                    split+=25
            # pd.DataFrame(l3, columns=[['count','emotion']]).to_csv('/Users/heemin/Documents/DHLab/mv/project/dir/emotion_top3/'+j[1])
            video_second_total = 0
            video_second_correct = 0
            for scound, emotion in l3:
                video_second_total += 1
                n_total+=1
                if emotion_dict[emotion] == label:
                    n_correct += 1
                    video_second_correct += 1
                # print(file, emotion, label)
            
            video_second_acc = video_second_correct / video_second_total
            # print(video_second_acc)
            n_video_total += 1
            if video_second_acc >= threshold:
                n_video_correct+=1
    n_video_acc = n_video_correct / n_video_total
    e_video_acc = e_video_correct / e_video_total
    video_acc = (e_video_correct+n_video_correct) / (e_video_total+n_video_total)

    n_acc = n_correct / n_total
    e_acc = e_correct / e_total
    acc = (n_correct+e_correct) / (n_total+e_total)
    print(f'threshold:{n}\nAccuracy for Neutral data: {n_acc*100:0.2f}%\nAccuracy for Emotion data: {e_acc*100:0.2f}%\nAccuracy for Total data: {acc*100:0.2f}%\nAccuracy by Neutral video: {n_video_acc*100:0.2f}%\nAccuracy by Emotion video: {e_video_acc*100:0.2f}%\nAccuracy by Total video: {video_acc*100:0.2f}%\n')

threshold:1
Accuracy for Neutral data: 50.24%
Accuracy for Emotion data: 76.91%
Accuracy for Total data: 56.27%
Accuracy by Neutral video: 56.12%
Accuracy by Emotion video: 89.32%
Accuracy by Total video: 70.25%

threshold:2
Accuracy for Neutral data: 55.23%
Accuracy for Emotion data: 75.45%
Accuracy for Total data: 59.80%
Accuracy by Neutral video: 64.03%
Accuracy by Emotion video: 85.44%
Accuracy by Total video: 73.14%

threshold:3
Accuracy for Neutral data: 59.85%
Accuracy for Emotion data: 72.55%
Accuracy for Total data: 62.72%
Accuracy by Neutral video: 66.91%
Accuracy by Emotion video: 82.52%
Accuracy by Total video: 73.55%

threshold:4
Accuracy for Neutral data: 64.37%
Accuracy for Emotion data: 70.36%
Accuracy for Total data: 65.72%
Accuracy by Neutral video: 74.10%
Accuracy by Emotion video: 80.58%
Accuracy by Total video: 76.86%

threshold:5
Accuracy for Neutral data: 67.45%
Accuracy for Emotion data: 67.64%
Accuracy for Total data: 67.49%
Accuracy by Neutral video: 77.70%
Ac

In [111]:
top3

[('Happy', 13), ('Neutral', 11), ('none', 1)]

In [108]:
l3

[]

In [19]:
top3

[('none', 18), ('Neutral', 7)]

In [11]:
l3

[[25, 'Neutral'], 'Neutral', 'Neutral', [100, 'Sad'], [125, 'Sad']]

In [392]:
e_acc

0.6993865030674846

In [393]:
e_total

652

In [394]:
e_correct

456

In [395]:
acc

0.359436045524036

In [396]:
correct

2116

In [397]:
total

5887

In [398]:
acc

0.359436045524036

In [399]:
correct

2116

In [400]:
total

5887

In [403]:
print(correct+e_correct)
print(total + e_total)
(correct+e_correct) /( total + e_total)

2572
6539


0.393332313809451